# Global Modelling

This script creates a global model of the virus outbreak, based on an extended SEIR model and a transport model for the global exchange of the people. Details: 

- The world is split into grid cells of 1 degree longitude by 1 degree latitude. Each cell represents the population in that geographical region.
- The model is temporally discretized in 1 day modelling steps.
- Epidemiological model with the states "susceptible" (S), "exposed" (E), "silent infected" (I0), "infected" (I1), "cured" (C), and "deseased" (D)
- Transport model with a constant exchange of people from grid cell to grid cell

In [ ]:
# import the libraries 
import numpy as np
import gdal
import matplotlib.pyplot as plt

## Definition of the grid

In [ ]:
lon = np.arange(-180, 180, 1)
lat = np.arange(-90, 90, 1)
LonGrid, LatGrid = np.meshgrid(lon, lat)

Define country belonging of the grid points

In [ ]:
xxx

Define the population density map and assign them to the grid

In [ ]:
# load the file
FileName = 'population_density\\gpw_v4_population_density_rev11_2020_1_deg.tif'
ds = gdal.Open(FileName)
geoMatrix = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
arr = band.ReadAsArray()
del ds, band

# get the details of the transform
ulX = geoMatrix[0]
ulY = geoMatrix[3]
xDist = geoMatrix[1]
yDist = geoMatrix[5]
rtnX = geoMatrix[2]
rtnY = geoMatrix[4]
Ny, Nx = arr.shape

# flip the map 
if yDist < 0:
    arr = np.flipud(arr)

# flag for the -inf valued cells for later
FlagInf = arr < -10000

# total population in the world
PopTotal = 8e9

# normalize the population to reflect the world population of 8 billion
Population = arr
Population[FlagInf == 0] = Population[FlagInf == 0] / np.sum(Population[FlagInf == 0]) * PopTotal

# show the population grid for visual check
plt.figure()
plt.imshow(np.log10(Population), vmin = 0, vmax = np.log10(np.max(Population)), origin="lower")

Define the health index per grid cell

In [ ]:
xxx

Define the government interventions per grid cell

In [ ]:
xxx

Check that the different matrices are consistent

In [ ]:
# grid and population matrix
assert np.all(LonGrid.shape == Population.shape), 'inconsistency in between Grid and Population!!'

## Definition of the epidemiological model

In [ ]:
def SEI0I1DCmodel(S,E,I0,I1,C,D, beta0, beta1, gamma0, gamma1, eps1, delta, a):
    
    # the entire population
    N = S + E + I0 + I1 + C + D
    
    # change of the susecptible
    dSdt = -beta0 * I0 * S / N - beta1 * I1 * S / N 
    
    # chance of the exposed 
    dEdt = beta0 * I0 * S / N  + beta1 * I1 * S / N - a * E 
    
    # change of the infected I0
    dI0dt = a * E - delta * I0 - gamma0 * I0
    
    # change of the infected I1
    dI1dt = delta * I0 - gamma1 * I1 - eps1 * I1
    
    # change of the cured
    dCdt = gamma0 * I0 + gamma1 * I1
    
    # change of the dead
    dDdt = eps1 * I1
    
    # update the states
    S = S + dSdt
    E = E + dEdt
    I0 = I0 + dI0dt
    I1 = I1 + dI1dt
    C = C + dCdt
    D = D + dDdt
    
    # return the updated states
    return S, E, I0, I1, C, D

## Definition of the transport model


Define the state update as a function of the state StateOld and the ExchangeMatrix

In [ ]:
def StateUpdate(StateOld, ExchangeMatrix):
    
    # get the relative distribution of people in each classes 
    StateRel = np.zeros(StateOld.shape)
    for ii in range(StateRel.shape[0]):
        StateRel[ii,:] = StateOld[ii,:] / np.sum(StateOld[ii,:])
    
    # compute the delta state
    dState = np.zeros(StateOld.shape)
    for ii in range(StateOld.shape[1]):
        dState[:,ii] = np.matmul(ExchangeMatrix, StateRel[:,ii])
    
    # define the new state
    StateNew = StateOld + dState 
    
    # return the new state
    return StateNew

Define the ExchangeMatrix describing the number of people transferring from 1 cell to the next in each time step

In [ ]:
xxx

## Simulate the evolution of the virus

Describe the total number of time steps for the simulation 

In [ ]:
Ttotal = 365

Initialize the model parameters 

In [ ]:
# beta: amount of susceptible getting exposed
beta0 = 0.2
beta1 = 0.2

# gamme: gamma: amount of infected getting cured
gamma0 = 0.1
gamma1 = 0.1

# eps: people dying from I1
eps1 = 0.01

# delta: people moving from I0 to I1
delta = 0.5

# a: incubation parameter with exponential distribution with parameter a (the incubation period is a^-1) --> exposed to I0
a = 0.1

Define a single iteration of the simulation

In [ ]:
def SimulateSingleStep():
    
    xxx

Define the initial state for all grid cells

In [ ]:
xxx

Run the simulation by looping through the simulation steps

In [ ]:
for ii in range(Ttotal):
    
    # show the progress
    print('Iterations ' + str(ii) + ' of ' + str(Ttotal) + ' days...')
    
    # update the state
    
    # report on the new state
    
    # store the new state
    

## Calibration??

to be done...